In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from pymongo import MongoClient

To add a new book to the Mongo database, follow the code below. Then, either run the cleaning and lemmatizing functions on the entire database or load a pickle file with the lemmatized texts.

In [8]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

In [112]:
list(collection.find({}, {'id':1, 'title':1}))

[{'_id': '1342', 'title': 'Pride and Prejudice'},
 {'_id': '84', 'title': 'Frankenstein'},
 {'_id': '219', 'title': 'Heart of Darkness'},
 {'_id': '1080', 'title': 'A Modest Proposal'},
 {'_id': '98', 'title': 'A Tale of Two Cities'},
 {'_id': '15399', 'title': 'The Importance of Being Earnest'},
 {'_id': '76', 'title': "Alice's Adventures in Wonderland"},
 {'_id': '2148', 'title': 'Moby Dick; or The Whale'},
 {'_id': '58915', 'title': 'Voyage to Jupiter'},
 {'_id': '160', 'title': 'The Adventures of Sherlock Holmes'},
 {'_id': '2097', 'title': 'The Souls of Black Folk'},
 {'_id': '5200', 'title': 'Metamorphosis'},
 {'_id': '1952', 'title': 'The Yellow Wallpaper'},
 {'_id': '58912', 'title': 'The Earth Quarter'},
 {'_id': '2600', 'title': 'War and Peace'},
 {'_id': '2591', 'title': "Grimms' Fairy Tales"},
 {'_id': '345', 'title': 'Dracula'},
 {'_id': '1260', 'title': 'Jane Eyre'},
 {'_id': '174', 'title': 'The Picture of Dorian Gray'},
 {'_id': '8800', 'title': 'Walden, and On The Duty

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## Mongo single book import

In [10]:
def book_to_mongo(book_num):

    book_details = {}
    book_details['_id']  = str(book_num)
    driver = webdriver.Chrome(chromedriver)
    try:
        driver.get('https://www.gutenberg.org/ebooks/%s' % book_num)
        book_text = driver.find_element_by_link_text('Read this book online: HTML')
        #     book_text=driver.find_element_by_xpath('//*[@id="download"]/div/table/tbody/tr[8]/td[2]/a')
        book_text.click()
        time.sleep(2)
        pre_mat = driver.find_elements_by_tag_name('pre')
      
        title_search = re.compile('Title:(.*?)\\n')
        title = title_search.search(str(pre_mat[0].text))
        print('hello')
        title = title.group(0)
        
        just_title = re.compile(':(.*)')
        book_title = just_title.search(title)
        title = book_title.group(1).strip()
        print(title)
        book_details['title'] = title


        author_search = re.compile('Author:(.*?)\\n')
        author = author_search.search(str(pre_mat[0].text))
        author = str(author.group(0))
        just_name = re.compile(':(.*)')
        author_name = just_name.search(author)
        author = author_name.group(1).strip()
        book_details['author'] = author


        book_text = driver.find_elements_by_tag_name('p')
        text_for_dict = ''
        for x in range(len(book_text)):
            text_for_dict += (book_text[x].text)
        book_details['text'] = text_for_dict

        print(book_details)
        driver.close()
        
        collection.insert_one(book_details)
    except:
        driver.close()
    print(book_details)
    return book_details

## Lemmatize a single book and add to list/dict

To add a lemmatized book to the pickle file, run the function below.

In [5]:
# %autoreload 1
# %aimport spacy_lemmer

import sys
sys.path.insert(0, '../src/features/')
sys.path.insert(0, '../src/data/')

import spacy_lemmer
from clean_book import book_cleaner

## Add to list

In [ ]:
def new_text_to_lem_pickle(book_num):
    with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
        lemmed_text = pickle.load(input_file)
    
    book_for_lemming = book_cleaner(book_num)
    spacy_lemmed_text = spacy_lemmer.spacy_lem_text(book_for_lemming)
    lemmed_text.append(spacy_lemmed_text)
    
    with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "wb") as output_file:
        pickle.dump(lemmed_text, output_file)


In [107]:
with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as read_file:
    lemmed_text = pickle.load(read_file)

book_dictionary = {} 
for _id, text in zip(book_nums, lemmed_text):
    book_dictionary[_id] = text

## Add to dictionary

In [87]:
def new_text_to_lemdict_pickle(book_num):
    with open('../data/processed/lem_dict_500000.pk', "rb") as input_file:
        lemmed_dict = pickle.load(input_file)
    
    book_for_lemming = book_cleaner(book_num)
    spacy_lemmed_text = spacy_lemmer.spacy_lem_text(book_for_lemming)
    book_num = str(book_num)
    lemmed_dict[book_num] = spacy_lemmed_text
    
    with open('../data/processed/lemmed_dict_500000.pk', "wb") as output_file:
        pickle.dump(lemmed_dict, output_file)

In [88]:
for book in books:
    new_text_to_lemdict_pickle(book)

2 B R 0 2 B 14293
12173
Second Variety 84305
68696
Five Weeks in a Balloon 499000
420274
Bones 317953
266296
The Black Arrow 430702
365966


Either run the full cleaning and lemmatizing code commented out below, or load the pickled lemmed_text file.  Lemmatization takes a very long time, so a pickled list of the lemmatized texts is loaded.

In [89]:
with open('../data/processed/lem_dict_500000.pk', "rb") as input_file:
    lemmed_text = pickle.load(input_file)

In [12]:
books_for_lemming = [book_cleaner(str(book)) for book in book_nums]
spacy_lemmed_text = [spacy_lemmer.spacy_lem_text(book) for book in books_for_lemming]

Pride and Prejudice 499000
Frankenstein 415792
Heart of Darkness 208333
A Modest Proposal 19173
A Tale of Two Cities 499000
The Importance of Being Earnest 116522
Alice's Adventures in Wonderland 133809
Moby Dick; or The Whale 499000
Voyage to Jupiter 389891
The Adventures of Sherlock Holmes 499000
The Souls of Black Folk 395540
Metamorphosis 117359
The Yellow Wallpaper 30931
The Earth Quarter 109426
War and Peace 499000
Grimms' Fairy Tales 499000
Dracula 499000
Jane Eyre 499000
The Picture of Dorian Gray 424422
Walden, and On The Duty Of Civil Disobedience 499000
The Count of Monte Cristo 499000
Leaves of Grass 8758
Autographs for Freedom 295429
Great Expectations 499000
Peter Pan 250581
Dubliners 364177
The Narrative of the Life of Frederick Douglass 217104
Pygmalion 184851
The Romance of Lust 499000
Ulysses 499000
Siddhartha 213768
A Christmas Carol 155526
Treasure Island 355349
Leviathan 499000
Legendary Tales of the Highlands (Volume 1 of 3) 312155
Wuthering Heights 499000
The Ess

In [95]:
with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
    lemmed_text = pickle.load(input_file)

Query the database for all of the book information.  The dictionary and lists will be used for identifying the books while clustering, comparing, and building the Flask app.

In [110]:
def get_book_info():    
    book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
    books_for_dict = {}
    book_nums = []
    book_titles = []
    book_authors = []
    for book in book_ids:
        book_nums.append((book['_id']))
        book_titles.append((book['title']))
        book_authors.append((book['author']))

        url = 'https://www.gutenberg.org/ebooks/%s' % book['_id']
        books_for_dict[book['title']]=[book['author'],url]
    
    return [books_for_dict, book_nums, book_titles, book_authors]

book_info = get_book_info()
books_for_dict, book_nums, book_titles, book_authors = (book_info[0],
                                                        book_info[1],
                                                        book_info[2],
                                                        book_info[3])

In [111]:
len(book_nums)

84

In [98]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

## tfidf vectorizor

In [67]:
tfidf= TfidfVectorizer(min_df = .1, max_df = .8, stop_words='english', ngram_range=(1,2))
# either use spacy_lemmed_text or 
doc_word = tfidf.fit_transform(lemmed_text)

In [48]:
# with open('../src/models/tf_idf.pk', 'wb') as write_file:
# #     pickle.dump(tfidf, write_file)

## truncatedSVD 

Taked the lemmed text and performed truncatedSVD modeling on the tfidf vectorized text.

In [70]:
lsa = TruncatedSVD(45)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.02583087, 0.0481496 , 0.04631311, 0.03600072, 0.02841531,
       0.02631714, 0.02421828, 0.02176713, 0.02072187, 0.01987594,
       0.01942194, 0.01864856, 0.01699652, 0.01620568, 0.01594792,
       0.01562151, 0.01542834, 0.01509642, 0.01501084, 0.01478494,
       0.01447481, 0.01434231, 0.01408817, 0.01395398, 0.01386721,
       0.01375255, 0.01339427, 0.01309221, 0.01270381, 0.0125903 ,
       0.01224011, 0.01185297, 0.01179806, 0.01170143, 0.01143602,
       0.01143246, 0.01131784, 0.01110664, 0.01090381, 0.0107124 ,
       0.01059397, 0.01039968, 0.01003791, 0.00998762, 0.0095538 ])

In [71]:
sum(lsa.explained_variance_ratio_)

0.7621069848930564

In [57]:
with open('../src/models/lsa.pk', 'wb') as write_file:                                                               
    pickle.dump(lsa, write_file)
    

In [58]:
with open('../src/models/doc_lsa.pk', 'wb') as write_file:                                                           
    pickle.dump(doc_topic, write_file)  

Create a list of lsa scores to plot on a graph against topic numbers. 

In [72]:
lsa_scores = []
topic_num = []
for i in range(0,100,2):
    lsa = TruncatedSVD(i)
    doc_topic = lsa.fit_transform(doc_word)
    lsa_scores.append(sum(lsa.explained_variance_ratio_))
    topic_num.append(i)
    print(sum(lsa.explained_variance_ratio_))

0
0.07398010081682634
0.15625935568612764
0.21092292266426205
0.2566911373632762
0.2972223478774481
0.3337555916272248
0.367710818931396
0.3987980925494428
0.42833773387610535
0.4586274820169066
0.4870215276741111
0.5154575712608945
0.5426766338499801
0.5693171170377916
0.5942941341370408
0.6191643875340078
0.6427050588892624
0.6657769760870523
0.6881373379279194
0.7105952928765842
0.7313739535356526
0.7523399324942704
0.7715372236253617
0.7913065338592346
0.8097917555609021
0.8275098012099837
0.8453427479680452
0.8627470251652377
0.8788336299559157
0.894651747523227
0.90972386743172
0.9241037250229795
0.9378333810869537
0.9508224409752889
0.9632391851285695
0.9746238048381518
0.9850832018701521
0.9937868519779408
1.0000000000000002
1.0000000000000007
0.9999999999999994
1.0000000000000002
0.9999999999999994
0.9999999999999999
0.9999999999999992
0.9999999999999992
1.0000000000000016
1.0
1.0000000000000007


In [22]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
plt.title('SVD Explained variance reaches 1.0\nwhen topics reach number of books')
plt.xlabel('Number of Topics')
plt.ylabel('Explained Variance Ratio')
plt.plot(topic_num, lsa_scores)
plt.axvline(x = 76, color = 'purple', linestyle='--')
plt.text(73,-.12, 76, color ='r')
plt.show()
# plt.savefig('../reports/SVD_explained_variance.svg')

<Figure size 640x480 with 1 Axes>

In [73]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, tfidf.get_feature_names(), 10)


Topic  0
sir, lady, sister, gentleman, thou, daughter, marry, sea, glance, hat

Topic  1
thou, thee, hath, thy, sister, government, liberty, society, slave, honour

Topic  2
thou, thee, thy, hath, ship, ye, sea, doth, unto, forest

Topic  3
thou, thee, thy, sister, lady, sir, doth, gentleman, unto, thou art

Topic  4
glen, hill, stench, farmhouse, diary, horror, telescope, manuscript, telephone, cattle

Topic  5
ship, glen, hill, sea, island, captain, boat, stench, farmhouse, shore

Topic  6
invisible, sir, parlour, mariner, revolver, policeman, hath, bandage, smash, gentleman

Topic  7
slave, slavery, ship, sister, ye, sir, gentleman, dollar, negro, sea

Topic  8
hath, sect, government, slave, king, tone, slavery, legislative, hill, kingdom

Topic  9
sister, dollar, invisible, tenement, forest, machine, planet, little man, mariner, parlour

Topic  10
car, job, tenement, dollar, train, gentleman, cent, boss, office, kingdom

Topic  11
princess, forest, king, soldier, slave, sister, br

In [25]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = book_titles,
             columns = lsa.singular_values_)
# Vt.sort_values('component_5', ascending=False))
Vt
# Vt.sort_values(3.9570467863251446, ascending=False)

,3.956634531296266,1.752863259941028,1.6487444419116897,1.3520833162205719,1.2364131609104114,1.2220412543791248,1.1748848863014931,1.1371047234374214,1.122654082952828,1.0883369146117556,...,0.660174968045411,0.6583460408935907,0.651486903370058,0.6389185517123253,0.6296352812138584,0.609026612090382,0.5824706087867821,0.5375896844856969,0.5349366645332871,0.44719155159754537
Pride and Prejudice,0.57523,0.57313,-0.29943,0.09333,0.00221,-0.12386,0.13045,-0.07903,0.10329,0.03167,...,-0.00105,-0.01223,-0.01759,-0.01440,0.00747,-0.01976,-0.15029,-0.06635,0.12243,0.35471
Frankenstein,0.55096,0.16213,0.17398,0.10768,-0.13459,0.05272,0.32319,0.06923,0.01725,-0.06787,...,-0.09613,-0.14248,-0.16381,-0.05392,-0.02828,-0.07136,0.09696,0.01413,-0.01080,0.01233
Heart of Darkness,0.45401,-0.24156,0.08094,0.05912,-0.09414,-0.04836,0.12927,-0.09166,-0.02511,0.13746,...,-0.01975,-0.01880,0.00731,-0.01826,0.02678,-0.00594,0.00270,-0.00188,0.00060,0.00009
A Modest Proposal,0.22111,0.12907,0.14169,0.05889,0.05878,-0.01100,-0.20837,-0.03563,0.19934,-0.28705,...,0.01886,0.03468,0.03454,0.01181,-0.00264,-0.00243,-0.01663,-0.01392,-0.00609,0.00007
A Tale of Two Cities,0.60571,-0.06035,-0.05758,0.03309,-0.05464,0.00321,-0.05181,0.23251,0.00849,-0.04928,...,-0.00698,-0.00146,-0.10911,0.00696,-0.06925,-0.01819,-0.03230,-0.01848,0.00442,-0.01007
The Importance of Being Earnest,0.27613,0.11495,-0.13779,-0.09273,0.00645,0.05311,-0.05712,-0.07139,-0.23174,0.10392,...,-0.00238,0.01613,-0.02066,-0.00995,0.01678,0.00643,-0.00415,0.00248,-0.00239,-0.00448
Alice's Adventures in Wonderland,0.34594,-0.15671,-0.19037,0.02551,0.20573,0.17029,-0.19842,-0.19794,-0.09490,0.02742,...,-0.03748,-0.00631,0.01126,0.00127,0.02049,-0.01375,-0.00497,-0.00418,-0.00016,0.00405
Moby Dick; or The Whale,0.48574,-0.23026,0.25792,0.40499,0.01777,-0.20027,0.07369,-0.07352,-0.19957,-0.02324,...,-0.01037,0.23099,-0.21706,0.29727,-0.06323,0.05589,-0.04492,-0.03530,-0.01258,-0.00140
Voyage to Jupiter,0.15917,-0.04341,0.11900,-0.11289,-0.26225,-0.07998,0.07926,-0.21190,0.03371,0.03321,...,-0.00795,0.00034,0.01214,0.00456,-0.00317,0.00997,-0.00564,0.00398,-0.00212,-0.00075
The Adventures of Sherlock Holmes,0.62856,-0.08477,-0.12722,-0.09346,-0.21617,-0.11598,-0.10983,0.16014,-0.04679,0.01850,...,0.20262,-0.23251,-0.07631,-0.00574,-0.11953,0.05234,0.01433,0.00761,-0.00892,-0.00544


## Test Book

Load a book that is not a part of the data trained above.  Transform it with the fit tfidf and the fit svd.
In the example below, I used a Ta-nehisi Coates article.

In [26]:
file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/gutenberg_recommender/data/external/becoming.txt'
title_of_test = 'becoming'

In [27]:
#The book_to_compare module opens a text file, cleans it (i.e. removes proper nouns and pronouns)
#then lemmatizes it with spaCy.
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)
print(lemmed_test)

5971
[' sound a little like a bad joke , do not ? what happen when a solitude - love individualist marrie an outgoing family woman who do not love solitude one bit?the answer , be guess , be probably the good and most sustaining answer to nearly every question arise inside a marriage , no matter who be or what the issue be : find way to adapt . if be in for ever , there ’ really no choice . ’s graduation photofacebook   ‘ have so much – an education , a healthy sense of self , a deep arsenal of ambition – and be wise enough to credit mother with instill in : of the - be to say that at the start of   , fly to and spend about five week live alone with thought while work on a draft of book dreams from , fill yellow legal pad with fastidious handwriting , distil idea during languid daily walk amid the coconut palm and lapping tide . , meanwhile , stay home on , live upstairs from mother , , as another leaden winter descend , shellac the tree and sidewalk with ice . keep busy , see friend a

In [28]:
test_tfidf = tfidf.transform(lemmed_test)
test_lsa = lsa.transform(test_tfidf)
print(test_lsa)

[[ 0.13772472  0.00550329 -0.00682855 -0.04495615  0.00298452 -0.01414102
  -0.01499137 -0.04474704 -0.00568138  0.00650008  0.0551554  -0.00445245
   0.00346174 -0.01024068 -0.00132332 -0.00189717 -0.00857951  0.00640245
   0.0044874  -0.00702552 -0.02169068 -0.00963187 -0.00675875  0.01098371
  -0.02737637 -0.01225982 -0.00904278 -0.01751015 -0.01451003  0.00125845
   0.01153075  0.01014039 -0.00299566  0.01005237  0.02372559 -0.00269307
  -0.02259426 -0.00472036 -0.00216738  0.00270029 -0.00077988 -0.02419953
  -0.01473561  0.0081298  -0.0004279  -0.00440992 -0.00792647  0.00176169
   0.00211161 -0.00755533 -0.00317738 -0.02977697 -0.00801395  0.00851689
  -0.01487931 -0.00211317 -0.02161104 -0.0024072   0.02507618 -0.00747837
   0.01747377  0.01119818 -0.0047203   0.00682327 -0.00162141 -0.01146336
  -0.00322756  0.01661031  0.00834641  0.01852632 -0.00310148  0.00327131
  -0.01780688 -0.01344457  0.01610401 -0.0081376   0.00292151 -0.00142156]]


Recompile book info variables just to make sure the information matches the lemmed_text variable.

In [29]:
book_info = get_book_info()
book_nums, book_titles, book_authors = (book_info[1],
                                        book_info[2],
                                        book_info[3])

In [30]:
book_titles.append(title_of_test)

In [31]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


Append the test_book transformed into the SVD tops to the array of topics of the training set
to enable cosine similarity and clustering.

In [32]:
doc_for_k = np.concatenate([doc_topic, test_lsa], axis=0)
doc_for_k

array([[ 5.75228811e-01,  5.73133319e-01, -2.99426570e-01, ...,
        -6.63461565e-02,  1.22432790e-01,  3.54710225e-01],
       [ 5.50961290e-01,  1.62125875e-01,  1.73984711e-01, ...,
         1.41313827e-02, -1.08040784e-02,  1.23288815e-02],
       [ 4.54008532e-01, -2.41556892e-01,  8.09439907e-02, ...,
        -1.88292311e-03,  5.96236962e-04,  9.38178913e-05],
       ...,
       [ 4.46298451e-01, -2.53643783e-01, -1.21398680e-01, ...,
        -1.28562728e-03,  1.39134370e-02,  3.96467944e-03],
       [ 3.47015947e-01, -1.89870807e-01,  9.65101900e-02, ...,
         3.46883025e-04,  3.14933108e-03,  1.24755233e-03],
       [ 1.37724723e-01,  5.50328932e-03, -6.82854722e-03, ...,
        -8.13760275e-03,  2.92151166e-03, -1.42156057e-03]])

Check out which topics are most strongly expressed in the test book

In [33]:
test_topics = doc_for_k[-1]
test_topics = np.argsort(test_topics)
print(test_topics)

[ 3  7 51 24 41 36 20 56 72 27  6 54 42 28  5 73 25 65 13 21 26 16 75 52
 46 49 59 19  2 22  8 37 62 11 45 66 50 70 32 35 57 38 55 15 64 77 14 40
 44 29 47 48 39 76  4 71 12 18  1 17  9 63 43 68 53 33 31 23 61 30 74 67
 60 69 34 58 10  0]


Find most similar books using cosine similarity.

In [34]:
a= cosine_similarity(doc_for_k)

In [35]:
top_cos_sim = np.argsort(a[-1])[:-1]
print(book_titles[top_cos_sim[-1]])
print(book_titles[top_cos_sim[-2]])
print(book_titles[top_cos_sim[-3]])

Little Women
The Jungle
Emma


In [36]:
num_clusters = 15
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=5) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=5, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [37]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

Frankenstein 6
A Tale of Two Cities 6
Voyage to Jupiter 6
War and Peace 6
Dracula 6
Jane Eyre 6
The Picture of Dorian Gray 6
Great Expectations 6
Peter Pan 6
Dubliners 6
Pygmalion 6
The Romance of Lust 6
A Christmas Carol 6
Legendary Tales of the Highlands (Volume 1 of 3) 6
Wuthering Heights 6
Songs of Innocence and Songs of Experience 6
Uncle Tom's Cabin 6
The Brothers Karamazov 6
Hard Times 6
The Time Machine 6
Little Women 6
Oliver Twist 6
Anne of Green Gables 6
The Adventures of Tom Sawyer, Complete 6
The Scarlet Letter 6
The Merry Adventures of Robin Hood 6
becoming 6


## NMF

In [38]:
from sklearn.decomposition import NMF

#retrain the tfidf vectorizor just to make sure it is fit on the training data.
tfidf= TfidfVectorizer(min_df = .1, max_df = .9, stop_words='english', ngram_range=(1,2))
doc_word = tfidf.fit_transform(spacy_lemmed_text)

In [39]:
nmf_model = NMF(7)
doc_topic = nmf_model.fit_transform(doc_word)

In [40]:
book_info = get_book_info()
book_nums, book_titles, book_authors = (book_info[1],
                                        book_info[2],
                                        book_info[3])
display_topics(nmf_model, tfidf.get_feature_names(), 20)


Topic  0
sister, dear, father, lady, mother, manner, brother, reply, family, pleasure, feeling, gentleman, general, letter, happy, marry, really, daughter, subject, attention

Topic  1
power, nature, opinion, subject, law, unto, government, common, wealth, desire, sect, society, action, case, truth, liberty, accord, public, whatsoever, manner

Topic  2
white, strange, black, window, earth, hill, moor, sky, case, human, star, paper, stone, brother, red, glass, darkness, glance, figure, road

Topic  3
ship, sea, captain, boat, island, shore, sail, voyage, kill, reply, king, board, till, cabin, squire, deck, mate, mast, master, doctor

Topic  4
slave, slavery, master, white, black, negro, nigger, mother, plantation, freedom, whip, land, labor, dollar, soul, law, nation, farm, school, liberty

Topic  5
tenement, dollar, car, job, train, pay, cent, street, money, city, family, boss, police, saloon, machine, bail, truck, tenant, floor, maybe

Topic  6
boy, girl, dear, mother, laugh, lady, f

In [41]:
#bring in the test book, and look at it's topic weights.  It expresses topic 2 most strongly. 
#Topic 2 is heavy in sci fi vocabulary, so it makes sense.

file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/gutenberg_recommender/data/external/three_body.txt'
title_of_test = '3b'
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)

test_tf = tfidf.transform(lemmed_test)
test_nmf = nmf_model.transform(test_tf)
test_nmf

43256


array([[0.01501148, 0.06427486, 0.15024117, 0.01405874, 0.03673038,
        0.06489087, 0.0609575 ]])

In [42]:
book_titles.append(title_of_test)

In [43]:
len(book_titles)

79

In [44]:
doc_for_k = np.concatenate([doc_topic, test_nmf], axis=0)

num_clusters = 10
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [45]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

A Modest Proposal 3
The Yellow Wallpaper 3
Leaves of Grass 3
Siddhartha 3
Les Misérables 3
My Secret Life, Volumes I. to III. 3
An Occurrence at Owl Creek 3
The Piebald Hippogriff 3
The Merry Adventures of Robin Hood 3
3b 3


In [46]:
doc_topic

array([[5.34096246e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.17006425e-01, 9.93634524e-02, 1.58975419e-01, 1.43126121e-01,
        5.52461747e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.30036979e-01, 1.75243252e-01,
        1.34901595e-02, 0.00000000e+00, 4.68683289e-02],
       [3.48875638e-02, 1.64973100e-01, 0.00000000e+00, 3.13845122e-02,
        0.00000000e+00, 2.13152073e-01, 0.00000000e+00],
       [1.26739886e-01, 2.74337963e-02, 1.96898576e-01, 7.98498530e-02,
        0.00000000e+00, 5.00154998e-02, 1.91063075e-01],
       [1.85187306e-01, 7.12317837e-03, 3.16328097e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.28741268e-01],
       [1.08390471e-03, 1.40669364e-03, 0.00000000e+00, 1.30421045e-02,
        0.00000000e+00, 0.00000000e+00, 3.88335574e-01],
       [0.00000000e+00, 1.64510321e-02, 1.03619561e-01, 6.01039206e-01,
        1.35539204e-02, 0

In [47]:
Vt = pd.DataFrame(doc_for_k.round(5),
             index = book_titles,
             columns = [0,1,2,3,4,5,6])
# Vt.sort_values('component_5', ascending=False))
# Vt.sort_values(1, ascending=False)
Vt

,0,1,2,3,4,5,6
Pride and Prejudice,0.53410,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Frankenstein,0.21701,0.09936,0.15898,0.14313,0.05525,0.00000,0.00000
Heart of Darkness,0.00000,0.00000,0.33004,0.17524,0.01349,0.00000,0.04687
A Modest Proposal,0.03489,0.16497,0.00000,0.03138,0.00000,0.21315,0.00000
A Tale of Two Cities,0.12674,0.02743,0.19690,0.07985,0.00000,0.05002,0.19106
The Importance of Being Earnest,0.18519,0.00712,0.00316,0.00000,0.00000,0.00000,0.12874
Alice's Adventures in Wonderland,0.00108,0.00141,0.00000,0.01304,0.00000,0.00000,0.38834
Moby Dick; or The Whale,0.00000,0.01645,0.10362,0.60104,0.01355,0.00000,0.00000
Voyage to Jupiter,0.00000,0.02073,0.20335,0.00000,0.00000,0.00000,0.00000
The Adventures of Sherlock Holmes,0.14179,0.00295,0.29890,0.00000,0.00000,0.13035,0.12743
